In [102]:
# import the relevant computational modules

# data manipulation
import pandas as pd #data processing
import numpy as np #linear algebra

# Models Packages
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.cross_validation import KFold

In [103]:
# import data

transaction_training = pd.read_csv('../raw_data/transactions_train.csv')
payment_training = pd.read_csv('../raw_data/paiements_train.csv')
billing_training = pd.read_csv('../raw_data/facturation_train.csv')
performance_training = pd.read_csv('../raw_data/performance_train.csv')


transaction_test = pd.read_csv('../raw_data/transactions_test.csv')
payment_test = pd.read_csv('../raw_data/paiements_test.csv')
billing_test = pd.read_csv('../raw_data/facturation_test.csv')
performance_test = pd.read_csv('../raw_data/performance_test.csv')

In [104]:
# Create basic scikit-learn wrapper model class
class SklearnWrapper:
    def __init__(self, clf, seed=0, params=None, seed_bool=True):
        if (seed_bool == True):
            params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict(x)

In [105]:
# create basic xgboost wrapper model class
class XgbWrapper:
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [106]:
# create basic lightGBM wrapper model class
class LightGbmWrapper:
    def __init(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 1550)
        self.verbose_eval = params.pop('verbose_eval', 100)
        
    def train(self, x_train, y_train):
        lgtrain = lgb.Dataset(x_train, y_train)
        self.lgbm = lgb.train(self.param, lgtrain, num_boost_round=self.nrounds, verbose_eval=self.verbose_eval)
    
    def predict(self, x):
        return self.lgbm.predict(lgb.Dataset(x))

In [107]:
# create out-of-fold predictions 
# make good use of k-fold CV's result 
# serving for the staking alogrithm 
# create a new column generated from model's score

def get_oof(clf, x_train, y, x_test):
    '''
    clf: the classifer, which can be logistic regression, SVM regression, Bayes classifier, etc.
    x_train: the training x in training dataset
    y: the training y in training dataset
    x_test: the testing x in training dataset 
    '''
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        x_te = x_train[test_index]
        
        clf.fit(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
    
    m = stats.mode(oof_test_skf, axis=1)
    oof_test[:] = m[0][0]
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)       

In [108]:
# data preprocess

class DataPreprocess:
    def __init__(self, label_encoder):
        self.lbl = label_encoder
        
    def preprocess_transcation(self, transaction_df):
        categorical_columns = ['MERCHANT_CATEGORY_XCD', 'MERCHANT_CITY_NAME', 'MERCHANT_COUNTRY_XCD', 
                               'DECISION_XCD', 'TRANSACTION_CATEGORY_XCD', 'TRANSACTION_TYPE_XCD', 'SICGROUP']
        
        for col in categorical_columns:
            transaction_df[col].fillna('unknown')
            transaction_df[col] = self.lbl.fit_transform(transaction_df[col].astype(str))
        
        transaction_df = transaction_df.groupby(['ID_CPTE', 'MERCHANT_CATEGORY_XCD'])['TRANSACTION_AMT'].sum()
        transaction_df = transaction_df.reset_index()
        transaction_df = transaction_df.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'MERCHANT_CATEGORY_XCD')
        transaction_df.columns = ['MERCHANT_CATEGORY_' + str(i) for i in transaction_df.columns]
        transaction_df = transaction_df.fillna(0)
        
        return transaction_df
    
    def preprocess_payment(self, payment_df):
        payment_df = payment_df.dropna()
        payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: str(x).split(' ')[0][:-3])
        payment_df = payment_df.sort_values(['ID_CPTE', 'TRANSACTION_DTTM'])
        payment_df['PAYMENT_N_COUNT'] = payment_df['PAYMENT_REVERSAL_XFLG'] == 'N'
        
        payment_df = payment_df.groupby(['ID_CPTE', 'TRANSACTION_DTTM'])[['TRANSACTION_AMT', 'PAYMENT_N_COUNT']].sum().reset_index()
        payment_df = payment_df.groupby('ID_CPTE').tail(12)
        
        tmp = payment_df.groupby(['ID_CPTE'])['PAYMENT_N_COUNT'].sum().reset_index()
        
        payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: x.split('-')[1])
        payment_df = payment_df.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'TRANSACTION_DTTM')
        payment_df.columns = ['transaction_' + str(i) for i in payment_df.columns + '_month']
        payment_df = payment_df.reset_index()
        payment_df = payment_df.fillna(0)
        
        payment_df = payment_df.merge(tmp, on='ID_CPTE')
        
        return payment_df
    
    def preprocess_billing(self, billing_df):
        billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[:-3])
        billing_df = billing_df.sort_values(['ID_CPTE', 'PERIODID_MY'])
        billing_df = billing_df.reset_index(drop=True)
        billing_df = billing_df.groupby('ID_CPTE').tail(12)
        billing_df = billing_df.reset_index(drop=True)
        billing_df['CreditLeft'] = billing_df['CreditLimit'] - billing_df['CurrentTotalBalance']
        
        billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[-2:])
        credit_left = billing_df.pivot_table('CreditLeft', ['ID_CPTE'], 'PERIODID_MY')
        credit_left.columns = ['credit_left_' + str(i) for i in credit_left.columns + '_month']
        cash_balance = billing_df.pivot_table('CashBalance', ['ID_CPTE'], 'PERIODID_MY')
        cash_balance.columns = ['cash_balance_' + str(i) for i in cash_balance.columns + '_month']
        
        delq_cycle = billing_df.groupby(['ID_CPTE'])['DelqCycle'].max().reset_index()
        delq_cycle = delq_cycle.rename(columns={'DelqCycle': 'MaxDelqCycle'})
        
        credit_left = credit_left.reset_index()
        cash_balance = cash_balance.reset_index()
        
        tmp = credit_left.merge(cash_balance, on='ID_CPTE')
        tmp = tmp.merge(credit_left, on='ID_CPTE')
        tmp = tmp.merge(delq_cycle, on='ID_CPTE')
        
        return tmp
    
    def merge(self, payment, billing):
        merge_df = payment.merge(billing, on='ID_CPTE', how='right')
        return merge_df.set_index(['ID_CPTE']) 

In [109]:
billing_test.head()

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
0,71424379,2013-11-01,2013-11-04,1444.30,0.0,3200.0,0
1,71424379,2014-05-01,2014-04-30,785.89,0.0,3200.0,0
2,71424379,2014-08-01,2014-08-02,1095.48,0.0,3200.0,0
3,71424379,2014-04-01,2014-04-02,845.30,0.0,3200.0,0
4,71424379,2013-12-01,2013-11-30,1623.28,0.0,3200.0,0


In [110]:
label_encoder = preprocessing.LabelEncoder()

In [111]:
preprocess = DataPreprocess(label_encoder)

In [112]:
processed_payment = preprocess.preprocess_payment(payment_training)

/home/data/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [113]:
processed_billing = preprocess.preprocess_billing(billing_training)

In [114]:
processed_data = preprocess.merge(processed_payment, processed_billing)

In [115]:
transaction_col = processed_data.iloc[:, :12].columns

In [116]:
for col in transaction_col:
    replace_value = processed_data[processed_data[col].notna()][col].mean()
    processed_data[col] = processed_data[col].fillna(replace_value)

In [117]:
processed_data = processed_data.fillna(0)

In [118]:
processed_data = processed_data.reset_index()

In [119]:
processed_data = processed_data.merge(performance_training[['ID_CPTE', 'Default']], on='ID_CPTE')

In [120]:
processed_data = processed_data.set_index('ID_CPTE')

In [121]:
X = np.array(processed_data.iloc[:, :-1])
y = np.array(processed_data.iloc[:, -1])

In [122]:
from imblearn.over_sampling import SMOTE

In [123]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, y)

In [150]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
    
clf = LogisticRegression()
rf = RandomForestClassifier(min_samples_split=200, max_depth=8, random_state=0)

In [151]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [152]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=200,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [153]:
predict_clf = clf.predict(X_test)

In [154]:
roc_auc_score(predict_clf, y_test)

0.7958241477168921

In [155]:
predict_rf = rf.predict(X_test)

In [156]:
roc_auc_score(predict_rf, y_test)

0.8473469607180288

In [73]:
processed_data[processed_data['Default'] == 1].head(20)

,transaction_01_month,transaction_02_month,transaction_03_month,transaction_04_month,transaction_05_month,transaction_06_month,transaction_07_month,transaction_08_month,transaction_09_month,transaction_10_month,...,credit_left_05_month_y,credit_left_06_month_y,credit_left_07_month_y,credit_left_08_month_y,credit_left_09_month_y,credit_left_10_month_y,credit_left_11_month_y,credit_left_12_month_y,MaxDelqCycle,Default
ID_CPTE,,,,,,,,,,,,,,,,,,,,,
10025534,131.300,0.00,260.00,0.00,6264.00,0.00,2080.00,0.00,318.00,0.00,...,4825.89,1550.35,817.41,-212.78,-301.85,-222.98,-169.94,-342.65,1,1
10081565,52.500,104.00,154.50,107.00,101.00,51.50,294.50,1408.95,289.70,292.00,...,164.74,220.96,205.66,1425.70,490.18,317.30,224.38,-2.86,0,1
10083607,510.000,207.00,1249.05,107.00,334.60,260.40,985.76,14.28,1121.70,1059.90,...,10.74,170.00,265.00,1100.00,339.08,158.68,79.40,289.39,0,1
10102455,420.000,202.00,434.22,336.27,790.68,632.24,55.62,205.50,0.00,1028.52,...,274.75,175.84,168.56,193.00,186.92,421.75,220.00,188.32,0,1
10205391,54.415,512.37,971.42,0.00,0.00,39.35,2110.19,2139.85,394.21,614.10,...,83.00,59.25,60.72,526.86,-13.02,-30.70,-18.12,9.90,1,1
10294646,721.000,11340.20,0.00,3717.80,515.00,1040.00,520.00,1369.00,835.00,0.00,...,-358.34,-453.36,2.16,-540.64,-1142.74,-785.78,-1629.28,-801.45,2,1
10303856,368.800,233.07,645.32,2614.22,719.06,1703.04,725.29,532.49,720.40,113.36,...,407.13,-58.72,387.44,-171.30,-141.24,-83.00,-66.56,-148.16,1,1
10351106,518.000,254.40,628.00,103.00,1053.00,983.00,41.60,150.00,308.10,250.24,...,2632.90,117.40,94.51,89.36,6.10,-174.88,16.60,878.27,2,1
10370887,0.000,0.00,0.00,65.27,206.00,0.00,320.30,4908.80,0.00,1010.00,...,2358.85,1338.46,1588.22,105.98,3234.08,3639.00,2094.54,1975.05,3,1


In [60]:
billing_training[billing_training['ID_CPTE'] == 86170782]

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
97534,86170782,2012-04,2012-04-16,14724.15,0.00,16100.0,0
97535,86170782,2012-11,2012-11-18,12116.58,1020.00,16100.0,0
97536,86170782,2012-08,2012-08-14,15938.00,0.00,16100.0,0
97537,86170782,2012-12,2012-12-13,13003.12,693.68,16100.0,0
97538,86170782,2011-12,2011-12-13,15292.41,0.00,16100.0,0
97539,86170782,2012-05,2012-05-15,16121.00,0.00,16100.0,0
97540,86170782,2012-03,2012-03-16,14796.60,0.00,16100.0,0
97541,86170782,2012-07,2012-07-17,10617.61,0.00,16100.0,0
97542,86170782,2012-09,2012-09-14,14403.90,918.00,16100.0,0
97543,86170782,2012-10,2012-10-19,9889.00,0.00,16100.0,0


In [36]:
from sklearn.externals import joblib
joblib.dump(clf, 'random_forest.pkl')

['random_forest.pkl']

In [136]:
processed_payment_test = preprocess.preprocess_payment(payment_test)
processed_billing_test = preprocess.preprocess_billing(billing_test)

/home/data/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [160]:
processed_test = preprocess.merge(processed_payment_test, processed_billing_test)

In [161]:
for col in transaction_col:
    replace_value = processed_test[processed_test[col].notna()][col].mean()
    processed_test[col] = processed_test[col].fillna(replace_value)

In [162]:
processed_test = processed_test.fillna(0)

In [163]:
processed_test.head()

,transaction_01_month,transaction_02_month,transaction_03_month,transaction_04_month,transaction_05_month,transaction_06_month,transaction_07_month,transaction_08_month,transaction_09_month,transaction_10_month,...,credit_left_04_month_y,credit_left_05_month_y,credit_left_06_month_y,credit_left_07_month_y,credit_left_08_month_y,credit_left_09_month_y,credit_left_10_month_y,credit_left_11_month_y,credit_left_12_month_y,MaxDelqCycle
ID_CPTE,,,,,,,,,,,,,,,,,,,,,
10074849,411.00,340.26,993.92,0.0,906.38,363.00,915.54,609.50,626.85,396.93,...,1082.89,1689.60,1355.5,1514.89,1691.76,1303.07,1254.59,1586.58,1237.53,0
10086539,0.00,556.92,832.00,642.0,661.26,1880.00,950.86,1591.00,1048.60,500.00,...,932.32,611.21,192.4,1309.15,845.20,601.42,394.90,866.74,865.76,0
10140908,214.10,88.36,200.00,206.0,239.99,160.50,418.00,428.72,202.00,163.20,...,66.75,388.00,305.1,616.10,726.48,942.60,917.25,543.06,8.44,2
10147994,38.11,39.52,218.40,227.9,224.70,229.69,226.00,504.29,4.24,256.20,...,1289.95,861.59,741.1,104.62,-52.00,-230.21,-374.16,158.00,-11.00,0
10152808,420.00,1030.00,510.00,423.3,877.50,1157.44,709.00,995.00,1015.00,515.00,...,800.69,571.52,775.0,957.00,1266.00,910.80,1132.00,674.48,818.62,0


In [64]:
processed_data[processed_data['Default'] == 0].head(20)

,transaction_01_month,transaction_02_month,transaction_03_month,transaction_04_month,transaction_05_month,transaction_06_month,transaction_07_month,transaction_08_month,transaction_09_month,transaction_10_month,...,credit_left_05_month_y,credit_left_06_month_y,credit_left_07_month_y,credit_left_08_month_y,credit_left_09_month_y,credit_left_10_month_y,credit_left_11_month_y,credit_left_12_month_y,MaxDelqCycle,Default
ID_CPTE,,,,,,,,,,,,,,,,,,,,,
10001822,318.00,522.50,374.50,4200.00,262.50,265.00,267.50,300.00,250.00,231.75,...,3420.58,2110.03,2184.77,1669.00,1246.00,967.72,-182.30,-432.20,0,0
10007972,784.34,168.28,1050.00,559.50,664.00,313.50,843.71,191.90,945.90,701.47,...,256.30,258.00,128.66,299.95,180.25,299.14,609.90,700.00,0,0
10012520,0.00,86.10,458.00,1177.31,315.00,525.00,505.00,50.50,1115.00,612.00,...,1580.97,1789.45,1704.09,1558.32,2323.32,2074.00,1606.10,1922.80,0,0
10033579,574.55,391.30,412.23,470.53,546.80,419.61,283.92,106.00,84.00,219.60,...,114.16,25.12,5.00,-24.28,-52.12,-38.21,-33.52,-129.33,1,0
10034823,982.03,0.00,2182.00,0.00,1180.69,0.00,1400.58,521.18,0.00,1977.00,...,725.00,1308.87,1390.95,1617.68,179.85,-296.68,1272.12,645.57,1,0
10036020,464.60,1483.30,739.54,0.00,2328.03,0.00,647.35,583.00,733.36,831.23,...,8176.09,8482.90,8477.26,8387.00,8220.46,8043.28,8518.60,8690.98,0,0
10068805,447.43,3336.17,3012.93,1564.34,2194.79,2740.56,3655.82,2660.58,1819.70,1737.91,...,9734.06,9411.96,9651.67,9431.00,10005.84,9360.68,10100.00,9662.16,0,0
10069450,83.70,473.58,569.61,303.22,1054.51,569.37,184.29,1007.32,176.44,200.39,...,1445.00,1490.80,1585.30,1234.35,1584.10,1518.10,1524.74,1283.06,0,0
10097162,303.00,1085.20,620.70,4097.19,283.00,247.22,120.00,1586.51,2410.26,1004.95,...,655.70,35.60,504.32,1099.36,3266.40,3198.00,2162.00,1693.00,0,0


In [164]:
X_test = np.array(processed_test)

In [165]:
processed_test['Default'] = rf.predict(X_test)

In [166]:
results = processed_test.reset_index()[['ID_CPTE', 'Default']]

In [167]:
results.to_csv('submission.csv')

In [97]:
results.head()

,ID_CPTE,Default
0,10074849,0
1,10086539,0
2,10140908,1
3,10147994,0
4,10152808,0


In [168]:
len(results)

5100

In [169]:
performance_test.head()

,ID_CPTE,PERIODID_MY,Default
0,71424379,2014-12-01,NaN
1,64887111,2015-12-01,NaN
2,69431075,2014-12-01,NaN
3,31823308,2016-12-01,NaN
4,39407834,2012-12-01,NaN
